In [6]:
import pandas as pd
import numpy as np
from src.insurer_model import Insurer
import matplotlib.pyplot as plt
import os

Scenerio 1-1:  natural hedging

Key Characteristics:
1. 
2. 
3. 
source: newspaper1.com, 

In [ ]:
my_insurer = Insurer(
    name="Typical Taiwanese Life Insurer",
    total_assets_twd=30_000_000_000_000, # Example: 30 Trillion TWD
    foreign_asset_ratio=0.70,
    usd_denom_ratio=0.95,
    initial_twd_usd_spot= , # Example start of period
    initial_us_bond_yield=0.045, # Example 4.5% annual yield
    initial_twd_equity=2_500_000_000_000, # Example 2.5 Trillion TWD
    initial_fevr_balance=220_000_000_000, # Example 220 Billion TWD
    bond_duration=8 # Example 8 years
)